# LLM Gateway 调试笔记本

本笔记本用于调试和测试 LLM Gateway 的各项功能：
- Provider 注册与管理
- 请求生成与响应处理
- Token 统计与成本追踪
- 限流机制测试

In [1]:
# 添加项目根目录到路径
import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

# 设置必要的环境变量（用于测试）
os.environ.setdefault("TELEGRAM_BOT_TOKEN", "test_token")
os.environ.setdefault("OPENAI_API_KEY", "your_openai_key_here")

'your_openai_key_here'

In [4]:
pip install anthropic


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/e5/70/7b0fd9c1a738f59d3babe2b4212031c34ab7d0fda4ffef15b58a55c5bcea/anthropic-0.76.0-py3-none-any.whl (390 kB)
Note: you may need to restart the kernel to use updated packages.


## 1. 导入 LLM Gateway 模块

In [5]:
from src.llm_gateway import (
    LLMGateway,
    LLMRequest,
    LLMResponse,
    ProviderConfig,
    OpenAIProvider,
    AnthropicProvider,
    VLLMProvider,
    RateLimiter,
    TokenBucket,
    TokenCounter,
    UsageStats
)

print("✅ LLM Gateway 模块导入成功")

✅ LLM Gateway 模块导入成功


## 2. 创建和配置 Gateway 实例

In [6]:
# 创建 Gateway 实例
gateway = LLMGateway(
    default_provider="openai",
    max_retries=3,
    retry_delay=1.0
)

print(f"Gateway 创建成功")
print(f"默认 Provider: {gateway.default_provider}")
print(f"最大重试次数: {gateway.max_retries}")

2026-01-21 15:15:57.747 | INFO     | src.llm_gateway.gateway:__init__:145 - LLM Gateway initialized


Gateway 创建成功
默认 Provider: openai
最大重试次数: 3


In [7]:
# 注册 OpenAI Provider（需要有效的 API Key）
# 如果没有真实的 API Key，可以跳过这个单元格

api_key = os.environ.get("OPENAI_API_KEY")

if api_key and api_key != "your_openai_key_here":
    config = ProviderConfig(
        api_key=api_key,
        model="gpt-4o-mini",  # 使用较便宜的模型进行测试
        max_tokens=100,
        temperature=0.7
    )
    
    provider = OpenAIProvider(config)
    gateway.register_provider("openai", provider)
    print("✅ OpenAI Provider 注册成功")
else:
    print("⚠️ 请设置有效的 OPENAI_API_KEY 环境变量")

⚠️ 请设置有效的 OPENAI_API_KEY 环境变量


## 3. 测试限流器 (Rate Limiter)

In [8]:
# 创建限流器
rate_limiter = RateLimiter(
    requests_per_minute=60,
    tokens_per_minute=100000,
    concurrent_requests=10
)

print(f"限流器配置:")
print(f"  - 每分钟请求数: {rate_limiter.requests_per_minute}")
print(f"  - 每分钟 Token 数: {rate_limiter.tokens_per_minute}")
print(f"  - 最大并发请求: {rate_limiter.concurrent_requests}")

限流器配置:
  - 每分钟请求数: 60
  - 每分钟 Token 数: 100000
  - 最大并发请求: 10


In [9]:
# 测试限流
import asyncio

async def test_rate_limit():
    """测试限流获取许可"""
    user_id = "test_user_001"
    
    # 连续请求几次
    for i in range(5):
        result = await rate_limiter.acquire(user_id=user_id, estimated_tokens=100)
        print(f"请求 {i+1}: {'✅ 通过' if result else '❌ 被限流'}")
    
    # 获取统计信息
    stats = rate_limiter.get_stats()
    print(f"\n限流统计: {stats}")

# 运行测试
await test_rate_limit()

请求 1: ✅ 通过
请求 2: ✅ 通过
请求 3: ✅ 通过
请求 4: ✅ 通过
请求 5: ✅ 通过

限流统计: {'total_requests': 5, 'rejected_requests': 0, 'active_users': 1}


## 4. 测试 Token 计数器

In [10]:
# 创建 Token 计数器
counter = TokenCounter()

# 模拟记录一些使用数据
usage1 = counter.record_usage(
    prompt_tokens=100,
    completion_tokens=50,
    model="gpt-4",
    provider="openai",
    user_id="user_001",
    bot_id="soulmate_bot"
)

print(f"记录使用 1:")
print(f"  总 Token: {usage1.total_tokens}")
print(f"  预估成本: ${usage1.cost:.6f}")

2026-01-21 15:16:12.790 | DEBUG    | src.llm_gateway.token_counter:record_usage:231 - Token usage recorded: 150 tokens, $0.006000 (model: gpt-4, user: user_001)


记录使用 1:
  总 Token: 150
  预估成本: $0.006000


In [11]:
# 再记录一些数据
usage2 = counter.record_usage(
    prompt_tokens=200,
    completion_tokens=100,
    model="gpt-4o-mini",
    provider="openai",
    user_id="user_001",
    bot_id="assistant_bot"
)

usage3 = counter.record_usage(
    prompt_tokens=150,
    completion_tokens=75,
    model="gpt-4",
    provider="openai",
    user_id="user_002"
)

print(f"记录使用 2: {usage2.total_tokens} tokens, ${usage2.cost:.6f}")
print(f"记录使用 3: {usage3.total_tokens} tokens, ${usage3.cost:.6f}")

2026-01-21 15:16:13.134 | DEBUG    | src.llm_gateway.token_counter:record_usage:231 - Token usage recorded: 300 tokens, $0.000090 (model: gpt-4o-mini, user: user_001)
2026-01-21 15:16:13.135 | DEBUG    | src.llm_gateway.token_counter:record_usage:231 - Token usage recorded: 225 tokens, $0.009000 (model: gpt-4, user: user_002)


记录使用 2: 300 tokens, $0.000090
记录使用 3: 225 tokens, $0.009000


In [12]:
# 查看统计信息
print("=" * 50)
print("总体统计:")
total_stats = counter.get_total_stats()
for key, value in total_stats.items():
    print(f"  {key}: {value}")

print("\n用户 user_001 统计:")
user_stats = counter.get_user_stats("user_001")
if user_stats:
    for key, value in user_stats.items():
        print(f"  {key}: {value}")

print("\n按模型统计:")
model_stats = counter.get_model_stats()
for model, stats in model_stats.items():
    print(f"  {model}: {stats['total_tokens']} tokens, ${stats['cost']:.6f}")

总体统计:
  prompt_tokens: 450
  completion_tokens: 225
  total_tokens: 675
  total_requests: 3
  total_cost: 0.01509

用户 user_001 统计:
  prompt_tokens: 300
  completion_tokens: 150
  total_tokens: 450
  requests: 2
  cost: 0.00609

按模型统计:
  gpt-4: 375 tokens, $0.015000
  gpt-4o-mini: 300 tokens, $0.000090


## 5. 测试 LLM 请求（需要有效的 API Key）

In [13]:
# 创建请求对象
request = LLMRequest(
    messages=[
        {"role": "system", "content": "你是一个友好的助手，请用简短的中文回复。"},
        {"role": "user", "content": "你好，今天天气怎么样？"}
    ],
    user_id="test_user",
    bot_id="test_bot",
    max_tokens=50
)

print(f"请求 ID: {request.request_id}")
print(f"消息数: {len(request.messages)}")
print(f"用户 ID: {request.user_id}")

请求 ID: 71a48c03-6421-46e3-b293-53e03c150e7c
消息数: 2
用户 ID: test_user


In [14]:
# 发送请求（需要有效的 Provider）
# 注意：这会实际调用 API，可能产生费用

if gateway.list_providers():
    response = await gateway.generate(request)
    
    print(f"响应成功: {response.success}")
    if response.success:
        print(f"内容: {response.content}")
        print(f"使用的 Token: {response.usage.total_tokens}")
        print(f"延迟: {response.latency_ms:.0f}ms")
    else:
        print(f"错误: {response.error}")
else:
    print("⚠️ 没有注册的 Provider，请先注册一个 Provider")

⚠️ 没有注册的 Provider，请先注册一个 Provider


## 6. Gateway 统计信息

In [15]:
# 获取 Gateway 统计
gateway_stats = gateway.get_stats()

print("Gateway 统计信息:")
print(f"  总请求数: {gateway_stats['total_requests']}")
print(f"  成功请求: {gateway_stats['successful_requests']}")
print(f"  失败请求: {gateway_stats['failed_requests']}")
print(f"  成功率: {gateway_stats['success_rate']:.2%}")
print(f"  已注册 Provider: {gateway_stats['registered_providers']}")

Gateway 统计信息:
  总请求数: 0
  成功请求: 0
  失败请求: 0
  成功率: 0.00%
  已注册 Provider: []
